# Preguntas Guía

Se desea evaluar qué factores influyen en la captura de un Pokemon teniendo en cuenta los
par ́ametros inmutables del mismo junto con la pokebola utlizada. Para ello se deberán generar
distintos gráficos que representen las respuestas a las siguientes preguntas guía:

1. Acerca de pokebolas:
a) Ejecutando la función 100 veces, para cada Pokemon en condiciones ideales
(HP: 100%, LVL 100) ¿Cuál es la probabilidad de captura promedio para cada po-
kebola?
b) ¿Es cierto que algunas pokebolas son más o menos efectivas dependiendo de propie-
dades intr ́ınsecas de cada Pokemon? Justificar.
Sugerencia: Comparar efectividad (success/total attemps) como proporci ́on de la
efectividad de la Pokebola b ́asica para cada Pokemon

2. Acerca del estado del Pokemon:
a) ¿Las condiciones de salud tienen algún efecto sobre la efectividad de la captura? Si
es así, ¿Cuál es más o menos efectiva?
b) ¿Cómo afectan los puntos de vida a la efectividad de la captura?
Sugerencia: Elegir uno o dos Pokemones y manteniendo el resto de los par ́ametros
constantes, calcular la probabilidad de captura para distintos HP %
c) ¿Qu ́e par ́ametros son los que m ́as afectan la probabilidad de captura?
d ) Teniendo en cuenta uno o dos pokemones distintos: ¿Qu ́e combinaci ́on de condiciones
(propiedades mutables) y pokebola conviene utilizar para capturarlos?
e) A partir del punto anterior, ¿ser ́ıa efectiva otra combinaci ́on de par ́ametros teniendo
en cuenta un nivel del pokemon más bajo (o más alto)?